### 2.	Создайте свой «мешок слов», например, набор из 20-25 отзывов о чем-либо, напишите краткое пояснение к данным.

In [1]:
reviews = [
    "The movie was amazing, with a very engaging plot.",
    "The restaurant's service was disappointing.",
    "This product completely exceeded my expectations!",
    "Too expensive, but the quality is decent.",
    "I’m very happy with my purchase and recommend it to everyone!",
    "The delivery issues ruined the experience.",
    "Great atmosphere and fantastic food!",
    "Not worth the hype. It’s just average.",
    "The customer support was extremely helpful.",
    "Highly recommend this to anyone looking for a good deal.",
    "The product broke within a week, very disappointing.",
    "Amazing visuals, but the storyline was weak.",
    "This is the best purchase I've made all year!",
    "The software has too many bugs, not worth the price.",
    "Fast delivery and great packaging, 10/10.",
    "Terrible quality, I want a refund.",
    "The actor's performance was breathtaking.",
    "It’s overpriced for what it offers.",
    "Perfect for gifting. My friend loved it.",
    "The app is user-friendly and works as advertised."
]


### 3.	Создайте словарь стоп-слов, примените стоп-слова к своему «мешку слов»

In [2]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# converting reviews to lowercase and removing stop words
processed_reviews = []
for review in reviews:
    words = review.lower().split()
    filtered_words = [word for word in words if word not in ENGLISH_STOP_WORDS]
    processed_reviews.append(" ".join(filtered_words))

print(f"Original reviews: {reviews[:3]}")  
print(f"\nProcessed reviews: {processed_reviews[:3]}")  


Original reviews:
['The movie was amazing, with a very engaging plot.', "The restaurant's service was disappointing.", 'This product completely exceeded my expectations!']

Processed reviews:
['movie amazing, engaging plot.', "restaurant's service disappointing.", 'product completely exceeded expectations!']


### 4.	Масштабируйте данные с помощью tf-idf

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

# creation of a tf-idf vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(processed_reviews)

print("Feature Names:", vectorizer.get_feature_names_out()[:10]) 
print("TF-IDF Matrix Shape:", tfidf_matrix.shape)

Feature Names: ['10' 'actor' 'advertised' 'amazing' 'app' 'atmosphere' 'average' 'best'
 'breathtaking' 'broke']
TF-IDF Matrix Shape: (20, 70)


### 5.	Использовать n-gram (1, 2, 3), pipeline(vectorizer, regressor) - ??? Добавить 

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

# creation of a pipeline with n-gram vectorizer and regressor
pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1, 3))),  # unigrams, bigrams, trigrams
    ("regressor", LinearRegression())
])
# 1 for pos, 0 for neg
labels = [1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1]
pipeline.fit(processed_reviews, labels)
new_reviews = ["The movie was boring and too long.", "Excellent customer service!"]
predictions = pipeline.predict(new_reviews)
print(f"Predictions for new reviews: {predictions}")


Predictions for new reviews: [0.72762621 0.51747426]


### 6.	Исследуйте коэффициенты модели

In [7]:
coefficients = pipeline.named_steps["regressor"].coef_
feature_names = pipeline.named_steps["tfidf"].get_feature_names_out()
# combining and sorting by importance
coef_df = sorted(zip(coefficients, feature_names), reverse=True, key=lambda x: abs(x[0]))
print("Top 10 most impactful features:")
for coef, feat in coef_df[:10]:
    print(f"{feat}: {coef:.4f}")


Top 10 most impactful features:
disappointing: -0.3487
quality: -0.3357
worth: -0.2866
it overpriced: -0.2367
it overpriced offers: -0.2367
offers: -0.2367
overpriced: -0.2367
overpriced offers: -0.2367
purchase: 0.2317
10: 0.2212


### 7.	Сделайте выводы по работе.

Лабораторная работа состояла из нескольких этапов, начиная с обработки текстовых данных и заканчивая созданием модели признаков и последующим анализом их важности. Всего было использовано двадцать отзывов, в которых была проведена предтекстовая очистка - преобразование в нижний регистр и удаление стоп-слов - чтобы улучшить текст, сохранив ключевые слова.
 

После обработки текста для оценки важности слов использовалась модель TF-IDF. Результаты показали, что из 70 отличительных признаков в матрице TF-IDF одни из самых употребляемых и ключевых слов в описании включают 'actor', 'amazing', 'atmosphere' и 'broke', что означает их релевантность для рецензий.

 
Затем был создан pipeline, состоящий из векторизации с использованием n-грамм - униграмм, биграмм и триграмм, и регрессии. Модель была обучена на метках (положительный или отрицательный отзыв), и после предсказания для новых отзывов была получена вероятность принадлежности к позитивной категории.

 
Важным шагом было определение коэффициентов модели. Результаты показали, что наибольшее влияние на предсказание нескольких целевых переменных оказало наличие определенных слов, связанных с качеством и стоимостью продукта, таких как ```disappointing``` - «разочаровывает» и ```quality``` - «качество», а также таких терминов, как «цена завышена» (```it overpriced```), «стоит» (```worth```) и «предлагает» (```offers```).


Результаты работы показывают, что проведенная предобработка и анализ признаков позволили успешно построить модель для классификации отзывов, а также выделить наиболее значимые слова для предсказания позитивных или негативных отзывов.